In [2]:
## Notebook para criação da base com as quantidades vendidas por período para mostrar nos dashs

##### Iniciando o Notebook Incluir no projeto o token para exposição na nuvem caso seja necessário

In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(spark.sparkContext, 'de02faf2-fbe4-466f-b8cd-44296f096a9d', 'p-25957a2b4ca812c11c01435cf691ea3068358702')
pc = project.project_context

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210530190625-0002
KERNEL_ID = e053e461-9a88-45b6-9f4a-8d28ee2261b7


**Importando algumas bibliotecas Utilizadas no projeto**

In [3]:
# Importando algumas bibliotecas
import sys
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

**Obter os dados e criar as tabelas temporárias para utilização com comandos SQL no spark**

In [4]:
dfcompra = spark.read.parquet(project.get_file_url("compra.snappy.parquet"))
dfcompraEntrega = spark.read.parquet(project.get_file_url("compraentrega.snappy.parquet"))
dfcompraSku = spark.read.parquet(project.get_file_url("compraentregasku.snappy.parquet"))
dflojista = spark.read.parquet(project.get_file_url("lojista.snappy.parquet"))
dfSKUcate = spark.read.parquet(project.get_file_url("skucategoria.snappy.parquet"))

In [5]:
# Criando a tabela temporaria
dfcompra.createOrReplaceTempView("compra")
dfcompraEntrega.createOrReplaceTempView("compraentrega")
dfcompraSku.createOrReplaceTempView("compraentregasku")
dflojista.createOrReplaceTempView("lojista")
dfSKUcate.createOrReplaceTempView("skucategoria")

In [10]:
spark.sql("SELECT 60 AS pub\
,CeSku.idskureferencia\
,Lj.idlojista\
,Lj.estado\
,CeSku.idcompraentrega\
,sc.iddepartamento\
,sc.nomedepartamento\
,sc.idsetor\
,sc.nomesetor\
,cast(CP.data as date) AS datacompra\
,cast((cast('2021-04-30' as date)-cast(CP.data as date)) as char(20)) AS diff_dias\
,cesku.idsku\
,cesku.valorvendaunidade\
,cesku.valorfrete \
FROM COMPRA CP \
JOIN compraentrega ce ON CP.idcompra = ce.idcompra AND cp.idbandeira = ce.idbandeira \
JOIN compraentregasku CeSku ON CE.idcompraentrega = Cesku.idcompraentrega \
AND ce.idbandeira = CeSku.idbandeira \
JOIN lojista Lj ON lj.idlojista = ce.idlojista AND lj.idbandeira = ce.idbandeira \
JOIN SKUCATEGORIA sc ON sc.idsku = CeSku.idsku \
WHERE lj.idlojista IN (10012,37450,92893,80333,64189) \
and cast(CP.data as date) >= cast('2021-04-30' as date) - 60 \
;").show()

+---+---------------+---------+------+---------------+--------------+--------------------+-------+--------------------+----------+----------------+----------+-----------------+----------+
|pub|idskureferencia|idlojista|estado|idcompraentrega|iddepartamento|    nomedepartamento|idsetor|           nomesetor|datacompra|       diff_dias|     idsku|valorvendaunidade|valorfrete|
+---+---------------+---------+------+---------------+--------------+--------------------+-------+--------------------+----------+----------------+----------+-----------------+----------+
| 60|        8154506|    37450|    SP|       96172423|            73|     Eletroportáteis|   2963|Chocolateira e Fo...|2021-03-12|1 months 18 days|   8154506|         254.9000|   25.9500|
| 60|     1505102152|    10012|    GO|       96269353|           836|          Automotivo|    836|          Automotivo|2021-03-16|1 months 14 days|1505102152|          39.0000|   31.6100|
| 60|     1501283149|    10012|    GO|      114314527|      

## Tabela temporária -- Cria a tabela gerada em dataframe para gravação em disco

In [11]:
dfTabelaSaida = spark.sql("SELECT 60 AS pub\
,CeSku.idskureferencia\
,Lj.idlojista\
,Lj.estado\
,CeSku.idcompraentrega\
,sc.iddepartamento\
,sc.nomedepartamento\
,sc.idsetor\
,sc.nomesetor\
,cast(CP.data as date) AS datacompra\
,cast((cast('2021-04-30' as date)-cast(CP.data as date)) as char(20)) AS diff_dias\
,cesku.idsku\
,cesku.valorvendaunidade\
,cesku.valorfrete \
FROM COMPRA CP \
JOIN compraentrega ce ON CP.idcompra = ce.idcompra AND cp.idbandeira = ce.idbandeira \
JOIN compraentregasku CeSku ON CE.idcompraentrega = Cesku.idcompraentrega \
AND ce.idbandeira = CeSku.idbandeira \
JOIN lojista Lj ON lj.idlojista = ce.idlojista AND lj.idbandeira = ce.idbandeira \
JOIN SKUCATEGORIA sc ON sc.idsku = CeSku.idsku \
WHERE lj.idlojista IN (10012,37450,92893,80333,64189) \
and cast(CP.data as date) >= cast('2021-04-30' as date) - 60 \
;")

## GRAVAÇÃO EM DISCO** Os próximos passos é para gravação da tabela gerada em disco e disponibilizar nos ativos

In [12]:
dfTabelaSaida.repartition(1).write.mode("overwrite").parquet("/home/spark/shared/tmp/compra_produto.parquet")

In [14]:
dfTabelaSaida.repartition(1).write.mode("overwrite").csv('/home/spark/shared/tmp/compra_produto.csv')

In [15]:
# Obtendo credenciais para disponibilizar nos ativos
credentials_1 = {
    'IAM_SERVICE_ID': 'iam-ServiceId-f8ddc357-a261-42a2-a6f2-44ac38d6cc50',
    'IBM_API_KEY_ID': '5y97tXSIMk-o11IIlG-qhORjxg4zQ6yFprxqB54ntyX9',
    'ENDPOINT': 'https://s3-api.us-geo.objectstorage.service.networklayer.com',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'projetoviagrupo6-donotdelete-pr-ehmgbn0j3d7dfc',
    'FILE': 'compra.csv'
}

cgsClient = ibm_boto3.client(service_name='s3',
    ibm_api_key_id = credentials_1['IBM_API_KEY_ID'],
    ibm_auth_endpoint="https://iam.ng.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3-api.us-geo.objectstorage.service.networklayer.com')

In [17]:
ls tmp/compra_produto.parquet/

part-00000-a412c337-13dd-4b20-a6dc-1c8c411e8f56-c000.snappy.parquet  _SUCCESS


In [18]:
cgsClient.upload_file(Filename='tmp/compra_produto.parquet/part-00000-a412c337-13dd-4b20-a6dc-1c8c411e8f56-c000.snappy.parquet',Bucket=credentials_1['BUCKET'],Key='compra_produto.parquet')

In [19]:
ls tmp/compra_produto.csv/

part-00000-4c790677-4187-4f04-8342-0199259926e3-c000.csv  _SUCCESS


In [20]:
cgsClient.upload_file(Filename='tmp/compra_produto.csv/part-00000-4c790677-4187-4f04-8342-0199259926e3-c000.csv',Bucket=credentials_1['BUCKET'],Key='compra_produto.csv')